<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

fatal: destination path 'semantic_features_detection' already exists and is not an empty directory.


In [ ]:
#!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100024&authkey=ADLJzehr2ENRggw" -O mapillary_vistas.zip

In [ ]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21101634&authkey=ABTiTiHVXMI06vU"  -O mapillary_prelim.h5

--2020-05-24 12:56:23--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21101634&authkey=ABTiTiHVXMI06vU
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mmhp0a.am.files.1drv.com/y4m2Y0MS0VbG4QBeaf_vSDk6P-Nu-wBOwtrdEtkLtd7NP7G_2INQ28DZ_lNX4ewOmpoqvpagLc8b69pbQPrUyYihlKvTi6SJx9Vb5rnFfb4Crx9oUdifJIaXsSLgJhx4ehT4nOq9TrJ36skVLIgcoGERlZO1QDoyTBunhexJocGH9F0z_3wdHY9OzmAWamWa1DAm-jHHNVh7My87qKmkUgi1A/mask_rcnn_mapvistas_0083.h5?download&psid=1 [following]
--2020-05-24 12:56:24--  https://mmhp0a.am.files.1drv.com/y4m2Y0MS0VbG4QBeaf_vSDk6P-Nu-wBOwtrdEtkLtd7NP7G_2INQ28DZ_lNX4ewOmpoqvpagLc8b69pbQPrUyYihlKvTi6SJx9Vb5rnFfb4Crx9oUdifJIaXsSLgJhx4ehT4nOq9TrJ36skVLIgcoGERlZO1QDoyTBunhexJocGH9F0z_3wdHY9OzmAWamWa1DAm-jHHNVh7My87qKmkUgi1A/mask_rcnn_mapvistas_0083.h5?download&psid=1
Resolving mmhp0a.am.files.1d

In [ ]:
#!unzip -qq /content/mapillary_vistas.zip 'mapillary_vistas/validation/*' -d /content

In [ ]:
#!unzip -qq /content/mapillary_vistas.zip 'mapillary_vistas/config.json' -d /content

In [ ]:
!pip install imgaug
!pip install Cython
!pip install pycocotools

In [ ]:
import os
os.chdir('/content/semantic_features_detection/samples/mapillary/cytools')
!python setup.py build_ext --inplace
os.chdir('/content/semantic_features_detection/Notebook')

running build_ext
copying build/lib.linux-x86_64-3.6/mask_tools.cpython-36m-x86_64-linux-gnu.so -> 


In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import imageio
%tensorflow_version 1.x
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Ignore Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
print(ROOT_DIR)
from mrcnn import utils
import mrcnn.model_felix as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/mapillary/"))  # To find local version
import mapillary_felix

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
print(MODEL_DIR)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
print('finished')

TensorFlow 1.x selected.
/content/semantic_features_detection
/content/semantic_features_detection/logs
finished


Using TensorFlow backend.


In [ ]:
class_names= ['Unlabeled', 'Bird', 'Ground Animal', 'Curb', 'Fence', 'Guard Rail', 
              'Barrier', 'Wall', 'Bike Lane', 'Crosswalk - Plain', 'Curb Cut', 'Parking', 
              'Pedestrian Area', 'Rail Track', 'Road', 'Service Lane', 'Sidewalk', 'Bridge', 
              'Building', 'Tunnel', 'Person', 'Bicyclist', 'Motorcyclist', 'Other Rider', 
              'Lane Marking - Crosswalk', 'Lane Marking - General', 'Mountain', 'Sand', 
              'Sky', 'Snow', 'Terrain', 'Vegetation', 'Water', 'Banner', 'Bench', 'Bike Rack',
              'Billboard', 'Catch Basin', 'CCTV Camera', 'Fire Hydrant', 'Junction Box', 'Mailbox',
              'Manhole', 'Phone Booth', 'Pothole', 'Street Light', 'Pole', 'Traffic Sign Frame', 
              'Utility Pole', 'Traffic Light', 'Traffic Sign (Back)', 'Traffic Sign (Front)',
              'Trash Can', 'Bicycle', 'Boat', 'Bus', 'Car', 'Caravan', 'Motorcycle', 'On Rails',
              'Other Vehicle', 'Trailer', 'Truck', 'Wheeled Slow', 'Car Mount', 'Ego Vehicle']
selected_classes = [34, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52]
print(len(selected_classes))
for class_num in selected_classes:
    print(class_names[class_num])

17
Bench
Billboard
Catch Basin
CCTV Camera
Fire Hydrant
Junction Box
Mailbox
Manhole
Phone Booth
Street Light
Pole
Traffic Sign Frame
Utility Pole
Traffic Light
Traffic Sign (Back)
Traffic Sign (Front)
Trash Can


In [ ]:
class InferenceConfig(mapillary_felix.mapvistas):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    NUM_CLASSES = len(selected_classes) + 1
config = InferenceConfig()
config.display()
batch_size = config.BATCH_SIZE


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                30
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE               

In [ ]:
# TRAIN_DIR = '/content/mapillary_vistas'
# # Training dataset
# dataset_train = mapillary.MapillaryDataset()
# dataset_train.load_vistas(dataset_dir=TRAIN_DIR, subset='training', class_ids=selected_classes)
# dataset_train.prepare()

# Validation dataset
VAL_DIR = '/content/drive/My Drive/mapillary_vistas/mapillary_vistas'

dataset_val = mapillary_felix.MapillaryDataset()
dataset_val.load_vistas(dataset_dir=VAL_DIR, subset='validation', config=config, class_ids=selected_classes)
dataset_val.prepare()

class_id, class_name: 1, Bench
class_id, class_name: 2, Billboard
class_id, class_name: 3, Catch Basin
class_id, class_name: 4, CCTV Camera
class_id, class_name: 5, Fire Hydrant
class_id, class_name: 6, Junction Box
class_id, class_name: 7, Mailbox
class_id, class_name: 8, Manhole
class_id, class_name: 9, Phone Booth
class_id, class_name: 10, Street Light
class_id, class_name: 11, Pole
class_id, class_name: 12, Traffic Sign Frame
class_id, class_name: 13, Utility Pole
class_id, class_name: 14, Traffic Light
class_id, class_name: 15, Traffic Sign (Back)
class_id, class_name: 16, Traffic Sign (Front)
class_id, class_name: 17, Trash Can


In [ ]:
print(len(dataset_val.class_names))


18


In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="inference", config=config,
                          model_dir=MODEL_DIR)
model_path = '/content/mapillary_prelim.h5'
model.load_weights(model_path, by_name=True)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:
# Code from https://github.com/matterport/Mask_RCNN used and adapted

def compute_overlaps_masks(masks1, masks2):
    """Computes IoU overlaps between two sets of masks.
    masks1, masks2: [Height, Width, instances]
    """
    
    # If either set of masks is empty return empty result
    if masks1.shape[-1] == 0 or masks2.shape[-1] == 0:
        return np.zeros((masks1.shape[-1], masks2.shape[-1]))
    # flatten masks and compute their areas
    masks1 = np.reshape(masks1 > .5, (-1, masks1.shape[-1])).astype(np.float32)
    masks2 = np.reshape(masks2 > .5, (-1, masks2.shape[-1])).astype(np.float32)
    area1 = np.sum(masks1, axis=0)
    area2 = np.sum(masks2, axis=0)

    # intersections and union
    intersections = np.dot(masks1.T, masks2)
    union = area1[:, None] + area2[None, :] - intersections
    overlaps = intersections / union

    return overlaps

def trim_zeros(x):
    """It's common to have tensors larger than the available data and
    pad with zeros. This function removes rows that are all zeros.

    x: [rows, columns].
    """
    assert len(x.shape) == 2
    return x[~np.all(x == 0, axis=1)]


def compute_matches(gt_boxes, gt_class_ids, gt_masks,
                    pred_boxes, pred_class_ids, pred_scores, pred_masks,
                    iou_threshold=0.5, score_threshold=0.0):
    """Finds matches between prediction and ground truth instances.

    Returns:
        gt_match: 1-D array. For each GT box it has the index of the matched
                  predicted box.
        pred_match: 1-D array. For each predicted box, it has the index of
                    the matched ground truth box.
        overlaps: [pred_boxes, gt_boxes] IoU overlaps.
    """
    # Trim zero padding
    # TODO: cleaner to do zero unpadding upstream
    gt_boxes = trim_zeros(gt_boxes)
    gt_masks = gt_masks[..., :gt_boxes.shape[0]]
    pred_boxes = trim_zeros(pred_boxes)
    pred_scores = pred_scores[:pred_boxes.shape[0]]
    # Sort predictions by score from high to low
    indices = np.argsort(pred_scores)[::-1]
    pred_boxes = pred_boxes[indices]
    pred_class_ids = pred_class_ids[indices]
    pred_scores = pred_scores[indices]
    pred_masks = pred_masks[..., indices]

    # Compute IoU overlaps [pred_masks, gt_masks]
    overlaps = compute_overlaps_masks(pred_masks, gt_masks)

    # Loop through predictions and find matching ground truth boxes
    match_count = 0
    pred_match = -1 * np.ones([pred_boxes.shape[0]])
    gt_match = -1 * np.ones([gt_boxes.shape[0]])
    for i in range(len(pred_boxes)):
        # Find best matching ground truth box
        # 1. Sort matches by score
        sorted_ixs = np.argsort(overlaps[i])[::-1]
        # 2. Remove low scores
        low_score_idx = np.where(overlaps[i, sorted_ixs] < score_threshold)[0]
        if low_score_idx.size > 0:
            sorted_ixs = sorted_ixs[:low_score_idx[0]]
        # 3. Find the match
        for j in sorted_ixs:
            # If ground truth box is already matched, go to next one
            if gt_match[j] > -1:
                continue
            # If we reach IoU smaller than the threshold, end the loop
            iou = overlaps[i, j]
            if iou < iou_threshold:
                break
            # Do we have a match?
            
            match_count += 1
            gt_match[j] = i
            pred_match[i] = j
            break

    return gt_match, pred_match, overlaps

In [ ]:
# Compute the pmfs

class_names = ['BG'] + dataset_val.class_names[2:]

image_ids = np.random.choice(dataset_val.image_ids, 2000)
class_PMF = np.zeros((len(selected_classes), (len(selected_classes))))
for image_id in image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
          modellib.load_image_gt(dataset_val, config,
                                image_id, use_mini_mask=False)
    results = model.detect([image], verbose=0)
    r = results[0]
    # visualize.display_instances(image, boxes=r['rois'], masks=r['masks'], class_ids=r['class_ids'], class_names=class_names, scores=r['scores'])
    gt_match, pred_match, overlaps = compute_matches(gt_boxes=gt_bbox, gt_class_ids=gt_class_id, gt_masks=gt_mask, pred_boxes=r['rois'], 
                          pred_class_ids=r['class_ids'], pred_scores=r['scores'], pred_masks=r['masks'], iou_threshold=0.5)
    
    gt_match = list(map(int, gt_match))
    
    for i, matched_pred in enumerate(gt_match):
        if matched_pred == -1:  
            class_PMF[gt_class_id[i], 0] += 1 
        else:
            class_PMF[gt_class_id[i], r['class_ids'][matched_pred]] += 1 

    for i, matched_gt in enumerate(pred_match):
        if matched_gt == -1:
            class_PMF[0, r['class_ids'][i]] += 1 
for i, class_Pr in enumerate(class_PMF):
    summed = class_Pr.sum()
    if summed > 0:
        class_PMF[i,:] = class_PMF[i,:]/summed
print(class_PMF)

[[0.000e+00 4.244e+03 1.540e+02 3.900e+01 1.320e+02 3.300e+02 1.200e+01
  4.170e+02 0.000e+00 8.020e+02 9.622e+03 2.500e+02 2.716e+03 1.598e+03
  1.148e+03 2.529e+03 3.570e+02]
 [2.543e+03 1.524e+03 0.000e+00 0.000e+00 0.000e+00 6.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.100e+01 6.000e+00 0.000e+00 0.000e+00 2.700e+01
  3.900e+01 2.920e+02 1.200e+01]
 [2.100e+02 0.000e+00 5.200e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.400e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]
 [2.200e+01 0.000e+00 0.000e+00 2.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 4.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00]
 [2.700e+01 0.000e+00 0.000e+00 0.000e+00 2.600e+01 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 8.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00]
 [2.390e+02 2.000e+01 0.000e+00 1.000e+00 2.000e+00 8.800e+01 0.000e+00
  0.000e+00 0.000e+00 4.000e+00 0.000e+00 0

In [ ]:
# compute the pmfs
for i, class_Pr in enumerate(class_PMF):
    summed = class_Pr.sum()
    if summed > 0:
        class_PMF[i,:] = class_PMF[i,:]/summed
print(class_PMF)

[[0.00000000e+00 1.74291581e-01 6.32443532e-03 1.60164271e-03
  5.42094456e-03 1.35523614e-02 4.92813142e-04 1.71252567e-02
  0.00000000e+00 3.29363450e-02 3.95154004e-01 1.02669405e-02
  1.11540041e-01 6.56262834e-02 4.71457906e-02 1.03860370e-01
  1.46611910e-02]
 [5.70179372e-01 3.41704036e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.34529148e-03 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.46636771e-03 1.34529148e-03 0.00000000e+00
  0.00000000e+00 6.05381166e-03 8.74439462e-03 6.54708520e-02
  2.69058296e-03]
 [7.60869565e-01 0.00000000e+00 1.88405797e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 5.07246377e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [7.85714286e-01 0.00000000e+00 0.00000000e+00 7.14285714e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.42857143e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00

In [ ]:
#save pmfs as dict
class_names = ['BG'] + dataset_val.class_names[2:]
print(len(class_names))
PMF_dict = {}
for i, class_Pr in enumerate(class_PMF):
    PMF_dict[class_names[i]] = list(class_Pr)
print(PMF_dict)


17
{'BG': [0.0, 0.17429158110882956, 0.006324435318275154, 0.0016016427104722792, 0.00542094455852156, 0.013552361396303902, 0.0004928131416837782, 0.017125256673511294, 0.0, 0.03293634496919918, 0.3951540041067762, 0.01026694045174538, 0.1115400410677618, 0.06562628336755646, 0.047145790554414785, 0.10386036960985626, 0.014661190965092403], 'Billboard': [0.5701793721973094, 0.3417040358744395, 0.0, 0.0, 0.0, 0.0013452914798206279, 0.0, 0.0, 0.0, 0.002466367713004484, 0.0013452914798206279, 0.0, 0.0, 0.006053811659192825, 0.00874439461883408, 0.06547085201793722, 0.0026905829596412557], 'Catch Basin': [0.7608695652173914, 0.0, 0.18840579710144928, 0.0, 0.0, 0.0, 0.0, 0.050724637681159424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'CCTV Camera': [0.7857142857142857, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'Fire Hydrant': [0.43548387096774194, 0.0, 0.0, 0.0, 0.41935483870967744, 0.0, 0.0, 0.0, 0.0, 0.0, 0.129032

In [ ]:
import json
with open('/content/PMFs.json', 'w') as outfile:
    json.dump(PMF_dict, outfile)


In [ ]:
# compute the mAP
def compute_batch_ap(image_ids):
    APs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                              r['rois'], r['class_ids'], r['scores'], r['masks'])
        APs.append(AP)
    return APs

In [ ]:
# compute mAP for the whole validation set
image_ids = np.random.choice(dataset_val.image_ids, 2000)
APs = compute_batch_ap(image_ids)
print("mAP @ IoU=50: ", np.mean(APs))

In [ ]:
# compute the AP per class
def compute_batch_ap_classwise(image_ids):
    mAPs = np.zeros([len(selected_classes)])
    numAPs = np.zeros([len(selected_classes)])
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, config,
                                  image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        # visualize.display_instances(image, boxes=r['rois'], masks=r['masks'], class_ids=r['class_ids'], class_names=dataset_val.class_names[1:], scores=r['scores'])
        # visualize.display_instances(image, boxes=gt_bbox, masks=gt_mask, class_ids=gt_class_id, class_names=dataset_val.class_names[1:])

        for i, class_id in enumerate(range(17)):
            # Select one class
            mask = np.where(gt_class_id == class_id, True, False)
            gt_class_id_sel = gt_class_id[mask]

            gt_bbox_sel = gt_bbox[mask,:]
            gt_mask_sel = gt_mask[:,:,mask]
                            
            # Select one class
            mask = np.where(r['class_ids'] == class_id, True, False)
            r_rois_sel = r['rois'][mask,:]
            r_class_ids_sel = r['class_ids'][mask]
            r_scores_sel = r['scores'][mask]
            r_masks_sel = r['masks'][:,:,mask] 
            if len(gt_class_id_sel) != 0:
                AP, precisions, recalls, overlaps =\
                    utils.compute_ap(gt_bbox_sel, gt_class_id_sel, gt_mask_sel,
                                      r_rois_sel, r_class_ids_sel, r_scores_sel, r_masks_sel)
                print("precisions: {}".format(precisions))
                mAPs[i] = mAPs[i] + AP
                # print("mAPs: {}".format(mAPs))
                numAPs[i] += 1
                # print("numAPs: {}".format(numAPs))

    # print("mAPs: {}".format(mAPs))
    # print("numAPs: {}".format(numAPs))
    mAPs = mAPs / numAPs
    return mAPs

In [ ]:
image_ids = np.random.choice(dataset_val.image_ids,2000)
APs = compute_batch_ap_classwise(image_ids)
print("mAP @ IoU=50: ", APs)

precisions: [1. 1. 0.]
precisions: [0.5        0.5        0.5        0.33333333 0.25       0.2
 0.        ]
precisions: [1. 1. 0.]
precisions: [1. 1. 0.]
precisions: [1. 1. 0.]
precisions: [0.33333333 0.33333333 0.33333333 0.33333333 0.25       0.2
 0.16666667 0.14285714 0.        ]
mAP @ IoU=50:  [       nan        nan        nan        nan        nan        nan
        nan        nan        nan 1.         0.29166667        nan
 0.5               nan        nan 0.5               nan]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
for i in range(len(APs)):
    print("class name: {}: mAP: {}".format(class_names[i], APs[i]))
